## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
# Create X_df using your selected columns
X_df = attrition_df[['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'PercentSalaryHike', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']]

# Show the data types for X_df
print(X_df.dtypes)

Education                  int64
Age                        int64
DistanceFromHome           int64
JobSatisfaction            int64
PercentSalaryHike          int64
StockOptionLevel           int64
WorkLifeBalance            int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
NumCompaniesWorked         int64
dtype: object


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3, random_state=42)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
for col in X_df.select_dtypes(include=['object']).columns:
    X_df[col] = pd.to_numeric(X_df[col], errors='coerce')
X_df = X_df.convert_dtypes()

In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Create a OneHotEncoder for the Department column
department_train = y_train['Department']
department_test = y_test['Department']
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(department_train.values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = encoder.transform(department_train.values.reshape(-1, 1))
department_test_encoded = encoder.transform(department_test.values.reshape(-1, 1))

# Print arrays
print("Encoded Training Data:")
print(department_train_encoded)
print("Encoded Testing Data:")
print(department_test_encoded)

Encoded Training Data:
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Encoded Testing Data:
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [9]:
# Create a OneHotEncoder for the Attrition column
attrition_train = y_train['Attrition']
attrition_test = y_test['Attrition']
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(attrition_train.values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = encoder.transform(attrition_train.values.reshape(-1, 1))
attrition_test_encoded = encoder.transform(attrition_test.values.reshape(-1, 1))
train_subset = np.concatenate((attrition_train_encoded[:2], attrition_train_encoded[-2:]))
test_subset = np.concatenate((attrition_test_encoded[:2], attrition_test_encoded[-2:]))

# Print arrays
print("Encoded Training Data:")
print(np.array2string(train_subset, separator=', ', prefix='array([', suffix='])'))
print("Encoded Testing Data:")
print(np.array2string(test_subset, separator=', ', prefix='array([', suffix='])'))

Encoded Training Data:
[[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]]
Encoded Testing Data:
[[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.]]


## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
input_shape = X_train_scaled.shape[1]

# Create the input layer
inputs = Input(shape=(input_shape,))

# Create at least two shared layers
shared_layer_1 = Dense(64, activation='relu', name='shared1')(inputs)
shared_layer_2 = Dense(128, activation='relu', name='shared2')(shared_layer_1)
# shared_layer_3 = Dense(64, activation = 'relu')(shared_layer_2)
# shared_layer_4 = Dense(64, activation = 'relu')(shared_layer_3)
# shared_layer_5 = Dense(3, activation = 'relu')(shared_layer_4)
# shared_layer_6 = Dense(2, activation = 'relu')(shared_layer_5)

In [11]:
# Create a branch for Department
# with a hidden layer and an output layer
department = attrition_df['Department'].nunique()

# Create the hidden layer
department_hidden_layer = Dense(32, activation='relu', name='department_hidden')(shared_layer_2)

# Create the output layer
# department_output_layer = Dense(3, department, activation='softmax', name='department_output')(department_hidden_layer)
department_output_layer = Dense(5, activation='softmax', name='department_output')(department_hidden_layer)

In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer
# Create the hidden layer
attrition_hidden_layer = Dense(32, activation='relu', name='attrition_hidden')(shared_layer_2)

# Create the output layer
attrition_output_layer = Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden_layer)

In [13]:
# Create the model
model = Model(inputs=inputs, outputs=[department_output_layer, attrition_output_layer], name="model")

# Compile the model
# model.compile(optimizer=Adam(), loss=['categorical_crossentropy', 'binary_crossentropy'], metrics=['accuracy'])
model.compile(optimizer='adam', loss=['categorical_crossentropy', 'binary_crossentropy'], metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input_1[0][0]']             
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [14]:
# Create a label encoder object
encoder = LabelEncoder()

# Convert y_train and y_test to a 1-dimensional array if they are not already
y_train_1d = y_train.values.ravel() if y_train.ndim > 1 else y_train.values
y_test_1d = y_test.values.ravel() if y_test.ndim > 1 else y_test.values

# Fit the encoder to the training data and transform both the training and testing data
y_train_encoded = encoder.fit_transform(y_train_1d)
y_test_encoded = encoder.transform(y_test_1d)

# Now, convert the numeric labels to one-hot encoded format
y_train_one_hot = to_categorical(y_train_encoded, num_classes=5)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=5)

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
Accuracy is a commonly used metric for classification problems, but it might not always be the best choice, especially for imbalanced datasets or when false positives and false negatives have different costs. In such cases, other metrics like precision, recall, F1 score, or the area under the ROC curve (AUC-ROC) might be more informative. For multi-class classification problems, accuracy alone might not provide a comprehensive view of the model's performance, as it doesn't account for the distribution of classes in the dataset. Metrics like multi-class log loss or confusion matrix analysis can offer more insight into how well the model is performing across all classes.

2.
For the "Department" output layer, you chose a softmax activation function. This is appropriate because softmax is used for multi-class classification problems, as it outputs a probability distribution over the classes, which is suitable for your scenario where each sample can belong to one of multiple classes. For the "Attrition" output layer, you chose a sigmoid activation function followed by a binary cross-entropy loss. This setup is suitable for binary classification problems, where each sample belongs to one of two classes. The sigmoid function outputs a probability that the given input point belongs to a certain class, and the binary cross-entropy loss measures the error between the model's predictions and the true labels 3.

3.
**Feature Engineering**: Creating new features or transforming existing ones can sometimes improve model performance. This could involve domain-specific knowledge or exploratory data analysis to identify potentially useful features.

**Hyperparameter Tuning**: Experimenting with different hyperparameters, such as the learning rate, batch size, number of layers, and number of units per layer, can lead to performance improvements. Techniques like grid search or random search can be used for this purpose.

**Regularization**: Techniques like L1, L2 regularization, or dropout can help prevent overfitting by adding a penalty to the loss function or randomly dropping out neurons during training.
Ensemble Methods: Combining predictions from multiple models can often yield better performance than any single model. Techniques like bagging, boosting, or stacking can be explored.

**Data Augmentation**: For certain types of data (e.g., images), creating modified versions of the training data (e.g., rotations, translations) can help the model generalize better to unseen data.

**Using a Different Model Architecture**: Experimenting with different model architectures (e.g., convolutional neural networks for image data, recurrent neural networks for sequential data) can sometimes lead to better performance, depending on the nature of the data and the problem.
*Provided by Phind*